# Tests módulo `ariaclinprot`

In [1]:
import ariaclinprot as acp

## Prescripciones

In [41]:
pvdf, ccdf, oardf = acp.parse_prescription('../../prescripciones/ORLadyuvante.csv')

In [37]:
pvdf

,Volume,Dose,FxDose
0,PTV 63,63.000,2.100
1,PTV 54,54.000,1.800


In [42]:
ccdf

,Volume,Min,Max,AtLeast,NoMore
0,PTV 63,,,"[95, 95, 59.85]","[5, 105, 67.41]"
1,PTV 54,,,"[95, 95, 51.3]","[5, 105, 57.78]"


In [39]:
oardf

,Organ,Dmean,Dmax,DosimPars
0,Parotidas,26 Gy,,[V30$50%]
1,Tronco cerebral,,,"[D53$100%, V60$30 %]"
2,Medula espinal,,45 Gy,[]
3,Laringe,,70 Gy,[]
4,Mandibula,,70 Gy,[]
5,Tiroides,,45 Gy,[]
6,Cerebro sano,,,[V 60$30 %]
7,Oido interno,,50 Gy,[]
8,Nervios opticos,,50 Gy,[]
9,Quiasma optico,,50 Gy,[]


In [14]:
oardf.Organ.str.len()

0     16
1     13
2      6
3     15
4     18
5     20
6     20
7     22
8      7
9      9
10     8
11    14
12    18
13    12
14    20
15    17
16    21
17    25
Name: Organ, dtype: int64

## Escritura de protocolos clínicos

In [43]:
# Read protocol template
bbx = acp.parseProt('BareBone.xml')
# Preview
ProtocolID = 'ArtificialHN'
TreatmentSite = 'Head and Neck'
acp.modPreview(bbx, ID=ProtocolID, TreatmentSite=TreatmentSite)
# Phases
PlanID='ORL'
FractionCount = int(float(pvdf.Dose[0])/float(pvdf.FxDose[0]))
acp.modPhase(bbx, ID=PlanID, vFractionCount=FractionCount)
# Structutures
for pv in pvdf.itertuples():
    acp.addStructure(bbx, structureName=pv.Volume)
for oar in oardf.itertuples():
    acp.addStructure(bbx, structureName=oar.Organ)

# Plan objetives
for pv in pvdf.itertuples():
    ccVolumedf = ccdf[ccdf.Volume == pv.Volume]
    '''
    if not ccVolumedf.Min.str.match(' +')[0]:
        a=1
    if ccVolumedf.Max.str.match(' +')[0] == False:
        a=1
    '''
    if ccVolumedf.AtLeast.values[0]:
        atLeastlst = ccVolumedf.AtLeast.values[0]
        VolumePercentage =  atLeastlst[0]
        DosePercentage = float(atLeastlst[1])/100
        FxDoseGy = float(pv.FxDose) * DosePercentage
        DoseGy = float(pv.Dose) * DosePercentage
        acp.addPlanObjetive(bbx, ID=pv.Volume, vParameter=VolumePercentage,
                            vDose=FxDoseGy, vTotalDose=DoseGy, vModifier=0)
    if ccVolumedf.NoMore.values[0]:
        noMorelst = ccVolumedf.NoMore.values[0]
        VolumePercentage =  noMorelst[0]
        DosePercentage = float(noMorelst[1])/100
        FxDoseGy = float(pv.FxDose) * DosePercentage
        DoseGy = float(pv.Dose) * DosePercentage
        acp.addPlanObjetive(bbx, ID=pv.Volume, vParameter=VolumePercentage,
                            vDose=FxDoseGy, vTotalDose=DoseGy)
# Quality Indexes
for pv in pvdf.itertuples():
    ccVolumedf = ccdf[ccdf.Volume == pv.Volume]
    if ccVolumedf.AtLeast.values[0]:
        TreatmentDosePrescription = acp.getTreatmentDosePrescription(pvdf)
        atLeastlst = ccVolumedf.AtLeast.values[0]
        VolumePercentage =  atLeastlst[0]
        DosePercentage = float(atLeastlst[1])/100
        StructureRelativeDose = float(pv.Dose) * DosePercentage / TreatmentDosePrescription * 100
        acp.addQualityIndex(bbx, ID=pv.Volume, vType=2, vModifier=0, 
                            vValue=VolumePercentage, vTypeSpecifier=StructureRelativeDose, 
                            vReportDQPValueInAbsoluteUnits='false')
    if ccVolumedf.NoMore.values[0]:
        TreatmentDosePrescription = acp.getTreatmentDosePrescription(pvdf)
        noMorelst = ccVolumedf.NoMore.values[0]
        VolumePercentage = noMorelst[0]
        DosePercentage = float(noMorelst[1])/100
        StructureRelativeDose = float(pv.Dose) * DosePercentage / TreatmentDosePrescription * 100
        acp.addQualityIndex(bbx, ID=pv.Volume, vType=2, vModifier=1, 
                            vValue=VolumePercentage, vTypeSpecifier=StructureRelativeDose, 
                            vReportDQPValueInAbsoluteUnits='false')
# Write clincial protocol
acp.writeProt(bbx)

In [5]:
bbx = acp.parseProt('BareBone.xml')
TreatmentSite = 'Prostata + vvss hipofx'
acp.modPreview(bbx, ID='ArmaduraTest', TreatmentSite=TreatmentSite)
acp.modPhaseID(bbx, 'Bach')
structureName = 'Recto'
acp.addStructure(bbx, structureName)
acp.addPlanObjetive(bbx, ID=structureName, vParameter=85, vDose=1, vTotalDose=20, vModifier=11)
structureName = 'PTV'
acp.addStructure(bbx, structureName)
acp.addPlanObjetive(bbx, structureName, vParameter=95, vDose=3*.95, vTotalDose=60*.95, vModifier=0, vPrimary='true')
acp.addPlanObjetive(bbx, structureName, vParameter=5, vDose=3*1.07, vTotalDose=60*1.07)
structureName = 'Recto'
acp.addQualityIndex(bbx, ID= structureName, vType=5, vModifier=1, vValue=85, vTypeSpecifier=20, vReportDQPValueInAbsoluteUnits='true')
acp.writeProt(bbx, protout='TestSalida.xml')

In [10]:
import xml.etree.ElementTree as ET

In [11]:
px = ET.parse('Próstata 20Fx.xml')
ET.indent(px)
px.write('Próstata 20Fx.xml', encoding='utf-8', xml_declaration=True)

### Diccionario de estructuras

In [19]:
import pandas as pd

In [31]:
StructureNormalizationdf= pd.DataFrame([
    {'Name' : 'Parotida_D', 'OtherNames' : 'parótida derecha'},
    {'Name' : 'Parotida_I', 'OtherNames' : 'parótida izda'},
    {'Name' : 'Medula', 'OtherNames' : 'médula'},
    {'Name' : 'Medula', 'OtherNames' : 'médula'},
    {'Name' : 'Gland_lagrimal_D', 'OtherNames' : 'glandula lacrimal derecha, glandula lacrimal der, glandula lacrimal dcha, gland lacrimal derecha, gland lacrimal der, gland lacrimal dcha'},
    {'Name' : 'Gland_lagrimal_I', 'OtherNames' : 'glandula lacrimal izquierda, glandula lacrimal izq, glandula lacrimal izqda, gland lacrimal izquierda, gland lacrimal izq, gland lacrimal izqda'},
])
StructureNormalizationdf

,Name,OtherNames
0,Gland_lagrimal_D,"glandula lacrimal derecha, glandula lacrimal d..."
1,Gland_lagrimal_I,"glandula lacrimal izquierda, glandula lacrimal..."


In [32]:
StructureNormalizationdf.OtherNames.str.contains('glandula lacrimal derecha')

0     True
1    False
Name: OtherNames, dtype: bool

In [35]:
StructureNormalizationdf[StructureNormalizationdf.OtherNames.str.contains('glandula lacrimal derecha')].Name.values[0]

'Gland_lagrimal_D'